# Getting Started with Pyomo

[Pyomo](http://www.pyomo.org/) is a Python software package for modeling and solving optimization problems. Using Pyomo, a user creates an optimization model by specifying decision **variables**, **constraints**, and an optimization **objective**. Pyomo includes a rich set of features to enable modeling of complex systems. A user then uses Pyomo to do any necessary model transformations, specifies a solver, and computes a solution. Typically other graphics or data packages will be used to post-process the solution for display.

## Installing a Pyomo Development Environment

The first step in getting started with Pyomo is to install a development environment on your laptop. There are cloud services that could, in principle, provide a suitable environment. But unless such an environment has been set up specifically for your needs, it is generally easier and more productive to get started on your own laptop.

### Step 1. Install Anaconda

Developing scientific and engineering applications in [Python](https://www.python.org/) requires an interpreter for a particular version of the Python language, a collection of previously developed software libraries, and additional development tools including editors and package managers. Together these elements comprise a Python distribution.

There are many [Python distributions](https://wiki.python.org/moin/PythonDistributions) available from commercial and free sources.  The Anaconda distribution available from [Anaconda.com](https://www.anaconda.com/) is among the most complete and best known distributions currently available, and is available as a [free download](https://www.anaconda.com/download/) or in commercially supported enterprise version. Anaconda includes

* a Python interpreter,
* a user interface [Anaconda Navigator](https://docs.anaconda.com/anaconda/navigator/) providing access to software development tools,
* pre-installed versions of major python libraries,
* the [conda](https://conda.io/docs/index.html) package manager to manage python packages and environments. 

Installation Procedure:

1. If you have previously installed Anaconda and wish to start over, then a first step is to [uninstall the earlier version](https://docs.anaconda.com/anaconda/install/uninstall). While it is possible to maintain multiple versions of Anaconda, there are problems that can arise when installing new packages. Uninstalling prior installations of Anaconda installations is the easiest way to avoid those problems.

2. [Download](https://www.anaconda.com/download/) a version of Anaconda appropriate for your laptop. Unless you have a specific reason to use an earlier version of the Python language, download the 64-bit graphical installer for the latest version of Python (currently Python 3.6).

3. Locate and launch the graphical installer from your download directory. Either follow the prompts or consult these more [detailed instructions](https://docs.anaconda.com/anaconda/install/). Normally you will want to use the default choices to install Anaconda into your home folder (a.k.a. directory) for your use only. Generally there is no need to install the optional Microsoft VSCode. 

4. [Verify](https://docs.anaconda.com/anaconda/install/verify-install) that your installation is working. For example, you should be able to locate and lauch a new application [Anaconda Navigator](https://docs.anaconda.com/anaconda/navigator/).

5. Install any available package updates. Open a command line (either the Terminal application on a Mac located look in the Applications/Utilities folder, or the Command Prompt on Windows), and execute the following two commands on separate lines.


    conda update conda
    conda update anaconda
   
If everything is working correctly, these commands should download and install any recent updates to the Anaconda package.

### Step 2. Install Pyomo

The following commands install Pyomo and dependencies. These commands should be executed one at a time from a [terminal window on MacOS](https://www.wikihow.com/Open-a-Terminal-Window-in-Mac) or a [command window on Windows](https://www.digitalcitizen.life/7-ways-launch-command-prompt-windows-7-windows-8).

    conda install -c conda-forge pyomo
    conda install -c conda-forge pyomo.extras

### Step 3. Install Solvers

Solvers are needed to compute solutions to the optimization models developed using Pyomo.  The solvers [glpk](https://en.wikibooks.org/wiki/GLPK) (mixed integer linear optimization) and [ipopt](https://en.wikipedia.org/wiki/IPOPT) (nonlinear optimization) cover a wide range of optimization models that arise in process systems engineering and provide good starting point for learning Pyomo. These are installed with the following commands (again, executed one at a time from a terminal window or command prompt).  

    conda install -c conda-forge glpk
    conda install -c conda-forge ipopt
    
At this point you should have a working installation of a Python/Pyomo development environment.  If you're just getting started with Pyomo, at this point you should be able to use Anaconda Navigator to open a Jupyter session in a browswer, then download and open this notebook in Jupyter. If all's well, the Pyomo model in the cells below should produce useful results.

### Step 4. (Optional) Install Additional Solvers

The glpk and ipopt solvers are sufficient to handle meaningful Pyomo models with hundreds to several thousand variables and constraints. These solvers are available under open-source licenses

olving larger problems may wish to install a multi-threaded solver such as the COIN-OR [cbc](https://projects.coin-or.org/Cbc) solvers.

    conda install -c conda-forge coincbc

High performance commercial solvers, such as [Gurobi](http://www.gurobi.com/index) and [CPLEX](https://www.ibm.com/products/ilog-cplex-optimization-studio/pricing), are also available at no cost for many academic uses (this is a fantastic deal!), and with trial licenses for commercial use.

NOTE: If command window procedure fails on your laptop, you might try running these commands in a Jupyter notebook. In that case, each command needs to be prefixed with the shell escape `!`, and followed by the `-y` option to handle the y/n 
interaction. For example, the basic pyomo install command would read

    !conda install -c conda-forge pyomo -y

Each install command should be executed in a separate cell.

## Key Concepts in Pyomo

A typical Pyomo application involves the creation of at least two Pyomo objects from the following classes:

* **ConcreteModel()**  A python object representing the optimization problem to be solved.
* **SolverFactory()** A python object representing the mathematical progamming software to be used for calculating a solution.

There are a number of of open-source and commercial solvers that can be used with Pyomo. This simple structure allows one to test and find solvers suited to a particular application.  

A model, in turn, is composed of additional objects used to specify a problem. A minimal set of classes is needed to create useful models. These classes are:

* **Var()** Objects representing variables determined in the course of solving a particular problem.
* **Objective()** An object denoting the problem objective function that is to be either minimized or maximized.
* **Constraint()** Objects representing problem constraints.

The following cell presents a typical example.

## Example: Linear Production Model with Constraints

The mathematical formulation of a simple linear production model is given by

\begin{align}
\max_{x,y \geq 0} &\ 40\ x + 30\ y  & \mbox{objective}\\
\mbox{subject to:}\qquad \\
x & \leq 40  & \mbox{demand constraint} \\
x + y & \leq 80  & \mbox{labor A constraint} \\
2x + y & \leq 100 & \mbox{labor B constraint}
\end{align}

where $x$ and $y$ are the rates of production (in units per week) for two products.

#### Step 1. Import Pyomo.

The first step in any Pyomo project is to import relevant components of the Pyomo library. This can be done with the following python statement.

    from pyomo.environ import *
    
#### Step 2. Create the model object.

Pyomo provides two distinct methods to create models. Here we use `ConcreteModel()` to create a model instance which is appropriate when all of the data needed to complete the model is avaiable at the current time.

    model = ConcreteModel()
    
The Python variable `model` stores the model instance. This could be any valid Python variable name.
    
#### Step 3. Add the Decision Variables, Objective, and Constraints to the model object.

The first major component of a Pyomo model are decision variables which are added as fields to `model`. In the case we name the fields `model.x` and `model.y` corresponding to $x$ and $y$ in the process model. The Python class `Var()` is used to specify these as real numbers that must be greater than or equal to zero.

    model.x = Var(domain=NonNegativeReals)
    model.y = Var(domain=NonNegativeReals)
    
As we will see in other examples, the `domain` can specify other types of decision variables including reals, integers, and booleans.

The objective is specified as an algebraic expression involving the decision variables. Here we store the objective in `model.profit`, and use the optional keyword `sense` to specify a maximization problem.

    model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

Constraints are added as fields to the model, each constraint created using the `Constraint()` class. The constraints are specified using the `expr` keywork in the form of linear algebraic expressions of the decision variables.

    model.demand = Constraint(expr = model.x <= 40)
    model.laborA = Constraint(expr = model.x + model.y <= 80)
    model.laborB = Constraint(expr = 2*model.x + model.y <= 100)


#### Step 4.  Create a solver object and solve.

The Pyomo libary includes a `SolverFactory()` class used to specify a solver. In this case, because the problem is a linear programming problem, we use the `glpk` solver. 

    results = SolverFactory('glpk').solve(model)
    results.write()
    if results.solver.status == 'ok':
        model.pprint()
    
The `solve()` method attempts to solve the model using the specified solver, and returns `results` which can be inspected to see if, in fact, a solution was found. If a solution is found, then `model` will have a pretty-print method `pprint()` that creates a summary of the problem solution.

#### Step 5. Display the solution.

Most applications will require access to specific components of the model. If a solution is found, the model will include methods with the same name as the fields created above, and which can be used to access solution values.

    print('Profit = ', model.profit())

    print('\nDecision Variables')
    print('x = ', model.x())
    print('y = ', model.y())

    print('\nConstraints')
    print('Demand  = ', model.demand())
    print('Labor A = ', model.laborA())
    print('Labor B = ', model.laborB())

In [3]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(expr = 40*model.x + 30*model.y, sense=maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
results = SolverFactory('cbc').solve(model)
results.write()
if results.solver.status:
    model.pprint()

# display solution
print('\nProfit = ', model.profit())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

print('\nConstraints')
print('Demand  = ', model.demand())
print('Labor A = ', model.laborA())
print('Labor B = ', model.laborB())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -2600.0
  Upper bound: -2600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  Termination condition: optimal
  Error rc: 0
  Time: 0.03280496597290039
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
2

## Python Lists, Sets, Dictionaries, and Iterators

Pyomo is integrated with the Python language, and inherits significant functionality by leveraging the basic data structures of Python. To make the best use of Pyomo, it is important to understand the basics of Python lists, sets, and dictionaries.

### Data in Matrix/Vector Format

The example above used scalar modeling components, `model.x = Var()` and `model.y = Var()`, and each constraint was added as a separate line in the model.  This is fine for small problems with a just a few unknowns, but becomes impractical for larger problems.

Here we repeat the example above, this time using `numpy` arrays to enter the data. An indexed variable `model.x` represents the unknowns, and the constraints are indexed as well. The indices are represented by the Python `range()` statement.

In [2]:
from pyomo.environ import *
import numpy as np

# enter data as numpy arrays
A = np.array([[3, 4], [2, -3]])
b = np.array([26, -11])

# set of row indices
I = range(len(A))

# set of column indices
J = range(len(A.T))

# create a model instance
model = ConcreteModel()

# create x and y variables in the model
model.x = Var(J)

# add model constraints
model.constraints = ConstraintList()
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) == b[i])

# add a model objective
model.objective = Objective(expr = sum(model.x[j] for j in J))

# create a solver
solver = SolverFactory('glpk')

# solve
solver.solve(model)

# print solutions
for j in J:
    print("x[", j, "] =", model.x[j].value)

x[ 0 ] = 2.0
x[ 1 ] = 5.0
